

Laboratorio 3 - Modelo de Regresión Lineal

By Cristian Laynez - 201281

Se provee un dataset donde se muestran los precios de casas en King Country, Seattle. 

Este conjunto incluye los precios de las casas vendidas entre mayo 2014 y mayo 2015. 

El objetivo de este laboratorio es definir un modelo de regresion lineal donde se puedad verla relacion entre precio 
y pies cuadrasa del espacio habitable interior de los apartementos.

In [1]:
# Importar todas las librerias a utilizar
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
# --> Leer la data proporcionada
f = open('kc_house_data.csv', 'r')
reader = csv.reader(f)
header = next(reader, None)

# Se guardara la data en un diccionario
data = {}
for h in header:
    data[h] = []

# Se guardaran todos los valores
for row in reader:
    control = 0
    for h, v in zip(header, row):        
        d = float(v) if control >= 2 else v
        temp = [d]
        data[h].append(temp)
        control += 1


In [3]:
# Imprimir todos los headers que se encontraron
print(header)

['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [4]:
# Definir funcion lambda de costo
cost = lambda X, y, t : ((X @ t - y) ** 2).sum() / len(y)

# Definir funcion lambda de gradiente
grad = lambda X, y, t : 2 * X.T @ (X @ t - y) / len(y)

# Definir funcion lambda de normalizacion que nos ayudara a tener un sistema de regresion lineal mas optimo
norm = lambda v : np.sqrt((v ** 2).sum())

# ! Referencia y creditos al ejemplo proveido en clase :)

In [25]:
# --> Ajustando sistema de regresion lineal
def linear_regression(X, y ,t, cost, grad, a=0.1, n=100, on_step=None):
    costs = []
    for _ in range(n):
        nabla = grad(X, y, t)
        t -= a * nabla
        costs.append(norm(nabla))

        if on_step:
            on_step(t)

    return t, costs

# ! Referencia y creditos al ejemplo proveido en clase :)

In [26]:
DATASET_SET_SIZE = len(data['id'])
DATASET_SPARSE_RATIO = 20
DATASET_X_LIM = 10

In [31]:
# --> Obtener datos para mostrar la siguiente relacion:
# * precio y pies cuadrados del espacio habitable interior (price - sqft_living).
X = np.array(data['price']).reshape((DATASET_SET_SIZE, 1))
Xr = np.hstack((
    np.ones((DATASET_SET_SIZE, 1)),
    np.array(data['sqft_living']).reshape((DATASET_SET_SIZE, 1))
))
y = 3 + 2 * X + np.array(data['sqft_living']).reshape((DATASET_SET_SIZE, 1)) * DATASET_SPARSE_RATIO

to = np.random.rand(Xr.shape[1], 1)

In [32]:
# -> Utilizar la implementacion vectorial del algoritmo de regresion lineal
tf, costs = linear_regression(
    Xr, 
    y, 
    to, 
    cost, 
    grad, 
    a=0.025, 
    n=20,
)

print(f"Tf: {tf}")

Tf: [[-3.88947223e+107]
 [-9.66707119e+110]]


In [33]:
# Preparar las variables xm, xmr y ym
xm = np.array([[0], [DATASET_X_LIM]])
xmr = np.hstack((
    np.ones((2, 1)),
    xm
))
ym = xmr @ tf

In [40]:
# Se graficara el resultado del modelo de regresion lineal

# ! Graficar modelo de regresion lineal
plt.plot(Xr[:, 1], y, 'ro')
plt.plot(xm, ym)
plt.title("Regresion Lineal (price - sqft_living)")
plt.show()

# ! Graficar costos
plt.plot(costs)
plt.title("Costos")
plt.show()

--> Análisis sobre los hallazgos

...